### Importing Libraries

In [27]:
import time
import requests
import pandas as pd
from tqdm import tqdm 
import chromedriver_binary
from bs4 import BeautifulSoup
from selenium import webdriver

### Setting Up Browser

In [28]:
browser = webdriver.Edge()
browser.get('https://stockmages.netlify.app')

### Infinite Scroll to Reach the Bottom

In [29]:
for i in tqdm(range(0,5400000,1000)):
    browser.execute_script("window.scrollTo(0,"  + str(i) + ")")
    time.sleep(.1)

100%|██████████████████████████████████████████████████████████████████████████████| 5400/5400 [13:16<00:00,  6.78it/s]


### Scraping the Page

In [30]:
soup = BeautifulSoup(browser.page_source,'html.parser')

data = []
for sp in tqdm(soup.find_all('div', class_ = 'container')):
    img_link = sp.find('img').get('src')
    tags     = sp.find('span', class_ = 'tag-color').text[7:].strip()
    likes    = int(sp.find('div', class_ = 'likes-comments').find_all('span')[0].text.strip()[:-6])
    comments = int(sp.find('div', class_ = 'likes-comments').find_all('span')[1].text.strip()[:-9])
    data.append([img_link, tags, likes, comments])

100%|███████████████████████████████████████████████████████████████████████████| 9104/9104 [00:00<00:00, 10615.93it/s]


### Creating a Dataframe

In [31]:
df = pd.DataFrame(data, columns = ['img_link', 'tags', 'likes', 'comments'])

In [32]:
df.head()

,img_link,tags,likes,comments
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Clouds, Sky, Atmosphere, Blue Sky",196,55
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Ornithology, Hummingbird",76,20
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Rainbow, Rainfall, Subtropical",282,106
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Cherry Blossoms, Road, Japan, Sakura",42,11
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Cape Marguerite, Flower, Plant",39,15


In [33]:
df.describe()

,likes,comments
count,9104.000000,9104.000000
mean,97.483414,24.265158
std,200.092581,39.018250
min,0.000000,0.000000
25%,7.000000,2.000000
50%,28.000000,13.000000
75%,114.000000,32.000000
max,5214.000000,1126.000000


### Saving Dataframe in CSV Format

In [34]:
df.to_csv('Images.csv', index = False)

### 1) Downloading an Image

In [35]:
link = df['img_link'][0]
res = requests.get(link)

In [36]:
fd = open("Img.png", "wb")
fd.write(res.content)
fd.close()

### 2) Downloading All Images

In [37]:
def download(link, img_path ):
    res = requests.get(link)         
    fd = open(img_path,'wb')
    fd.write(res.content)            
    fd.close()

In [40]:
paths   = []
img_ids = []
ref = 100000
i   = 1

for link in tqdm(df['img_link']):
    img_id = 'A' + str(i + ref)[1:]          # Creating Image ID
    i += 1
    img_path = 'Imgs/' + img_id + '.jpg'    # Creating Image Path
    download(link, img_path)                # Downloading the Image
    paths.append(img_path)
    img_ids.append(img_id)

100%|██████████████████████████████████████████████████████████████████████████████| 9104/9104 [25:55<00:00,  5.85it/s]


In [41]:
df['img_id']   = img_ids
df['img_path'] = paths
df.head()

,img_link,tags,likes,comments,img_id,img_path
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Clouds, Sky, Atmosphere, Blue Sky",196,55,A00001,Imgs/A00001.jpg
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Ornithology, Hummingbird",76,20,A00002,Imgs/A00002.jpg
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Rainbow, Rainfall, Subtropical",282,106,A00003,Imgs/A00003.jpg
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Cherry Blossoms, Road, Japan, Sakura",42,11,A00004,Imgs/A00004.jpg
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Cape Marguerite, Flower, Plant",39,15,A00005,Imgs/A00005.jpg


In [42]:
df.to_csv('Imgs.csv', index = False)